In [8]:
import pickle
model_name="Montreal"
model_vars= pickle.load(open("../../models/"+model_name+".dat",'rb'))


XGBoostError: [01:38:25] /Users/travis/build/dmlc/xgboost/include/xgboost/json.h:65: Invalid cast, from Null to Array
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000011f68c1c0 dmlc::LogMessageFatal::~LogMessageFatal() + 112
  [bt] (1) 2   libxgboost.dylib                    0x000000011f6cab63 xgboost::JsonArray const* xgboost::Cast<xgboost::JsonArray const, xgboost::Value>(xgboost::Value*) + 739
  [bt] (2) 3   libxgboost.dylib                    0x000000011f79531f xgboost::RegTree::LoadModel(xgboost::Json const&) + 1519
  [bt] (3) 4   libxgboost.dylib                    0x000000011f724954 xgboost::gbm::GBTreeModel::LoadModel(xgboost::Json const&) + 548
  [bt] (4) 5   libxgboost.dylib                    0x000000011f714b53 xgboost::gbm::GBTree::LoadModel(xgboost::Json const&) + 419
  [bt] (5) 6   libxgboost.dylib                    0x000000011f72716d xgboost::LearnerIO::LoadModel(xgboost::Json const&) + 2173
  [bt] (6) 7   libxgboost.dylib                    0x00

In [ ]:

from alibi_detect.cd import TabularDrift
cd = TabularDrift(p_val=0.05, X_ref=X_ref, categories_per_feature={0: None, 3: None})
preds_drift = cd.predict(X, drift_type='batch', return_p_val=True, return_distance=True

In [10]:

import xgboost
import pickle

# load data

# split data into X and y
X = [0,1,2,3]
Y = [0,0,0,0]
# split data into train and test sets
seed = 7
test_size = 0.33
# fit model no training data
model = xgboost.XGBClassifier()

In [11]:
pickle.dump(model,open('model.pkl','wb'))

In [12]:
loaded_model = pickle.load(open("model.pkl", "rb"))


In [16]:
import pandas as pd
import numpy as np
import os
import platform as pf
pd.options.mode.chained_assignment = None  # default='warn'

#packages for time series
import warnings
import itertools    
import statsmodels.api as sm

#Richard was here
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import math
from scipy import stats
import click
import logging
from pathlib import Path
from dotenv import find_dotenv, load_dotenv



def train_test_split(data, n_test):
    return data[:n_test], data[n_test:]

def get_data(filename):
    df = pd.read_csv('../../data/processed/'+filename)
    return(df)


file_name="Montreal.csv"
df = get_data(file_name)
df['datetime'] = df['datetime'].astype('datetime64[ns]')


weather_df = df[['datetime', 'Temperature']].set_index('datetime')
ml_df = pd.DataFrame({'Temperature': weather_df['Temperature']}, index = weather_df.index)

# CREATE LAGGED ATTRIBUTES

ml_df['date'] = ml_df.index
ml_df['month'] = ml_df['date'].dt.day
ml_df['month'] = ml_df['date'].dt.month
ml_df['year'] = ml_df['date'].dt.year

ml_df['lag1'] = ml_df['Temperature'].shift(periods=1, fill_value=0)
ml_df['lag2'] = ml_df['Temperature'].shift(periods=2, fill_value=0)
ml_df['lag12'] = ml_df['Temperature'].shift(periods=12, fill_value=0)
ml_df['lag24'] = ml_df['Temperature'].shift(periods=24, fill_value=0)

ml_df['avg_returns'] = 0
for i in range(1, len(ml_df)):
    ml_df['avg_returns'][i] = ml_df['lag1'][:i + 1].mean()

ml_df.drop('date', axis=1, inplace=True)

# we have to eliminate all those with 0s
ml_df = ml_df[ml_df['lag24'] != 0]
X = ml_df.iloc[:,1:]
y = ml_df[['Temperature']]
split_len = int(len(weather_df)*0.80)

X_train, X_test = train_test_split(X, split_len)
y_train, y_test = train_test_split(y, split_len)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled)

import xgboost as xgb

xgb_model = xgb.XGBRegressor(n_estimators=1000)
xgb_model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_test, y_test)],
              early_stopping_rounds=50,
              verbose=False)

import pickle
print("DONE")
pickle.dump(xgb_model, open("../../models/"+model_name+".pkl", 'wb'))
pickle.dump(xgb_model, open("../../models/"+model_name+"transformer.pkl", 'wb'))


DONE


In [28]:

forecast = xgb_model.predict(X_test)

In [30]:
ml_df

,Temperature,month,year,lag1,lag2,lag12,lag24,avg_returns
datetime,,,,,,,,
2012-10-02 13:00:00,286.13686,10,2012,286.123721,286.110581,285.979185,285.830000,274
2012-10-02 14:00:00,286.15000,10,2012,286.136860,286.123721,285.992325,285.834650,274
2012-10-02 15:00:00,287.55000,10,2012,286.150000,286.136860,286.005465,285.847790,275
2012-10-02 16:00:00,288.14000,10,2012,287.550000,286.150000,286.018604,285.860929,275
2012-10-02 17:00:00,288.69000,10,2012,288.140000,287.550000,286.031744,285.874069,276
...,...,...,...,...,...,...,...,...
2017-11-29 20:00:00,275.13000,11,2017,276.450000,276.150000,275.380000,273.750000,280
2017-11-29 21:00:00,274.13000,11,2017,275.130000,276.450000,277.450000,274.050000,280
2017-11-29 22:00:00,273.48000,11,2017,274.130000,275.130000,278.770000,274.050000,280
